In [1]:
import warnings
warnings.filterwarnings('ignore',category=FutureWarning)
warnings.filterwarnings('ignore',category=DeprecationWarning)
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from functools import reduce
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

In [2]:
path = "../Our_Dataset"
files_list = []
for filename in os.listdir(path):
    f = os.path.join(path, filename)
        # checking if it is a file
    if os.path.isfile(f):
        files_list.append(f)
print("Number of files: {}".format(len(files_list)))

Number of files: 7


## Merge Dataset

In [25]:
i = 0
df = pd.DataFrame()
drop_list = ['Data','Hora', 'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5']
while i < len(files_list):
    print(files_list[i])
    if(files_list[i] == "../Our_Dataset/PT0002000052213942JN.xlsx"):
        i += 1
        continue
    data = pd.read_excel(files_list[i], sheet_name=['Total_Consumers'])
    df_tmp = data['Total_Consumers']
    df_tmp['Data'] = df_tmp['Data'].apply(lambda x: str(x).replace('/', '-'))
    df_tmp['Time'] = df_tmp['Data'] + ' ' + df_tmp['Hora']
    df_tmp['Time'] = pd.to_datetime(df_tmp['Time'], infer_datetime_format=True)
    tmp_filter = df_tmp.filter(drop_list)
    df_tmp.drop(tmp_filter, axis=1, inplace=True)
    df_tmp.dropna(subset=['Time'], inplace=True)
    df_tmp = df_tmp.iloc[:,[1,0]]
    #df_tmp.set_index('Time', inplace=True)
    #df = pd.concat([df, df_tmp], axis = 1)
    print(df_tmp.shape)
    #print("Time range" + ": " + str(min(df_tmp.index)) + " - " + str(max(df_tmp.index)))
    print("Time range" + ": " + str(min(df_tmp['Time'])) + " - " + str(max(df_tmp['Time'])))
    if len(df) == 0:
        df = df_tmp
    else:
        df = df.merge(df_tmp, on='Time', how='outer')
    df.rename(columns={'Consumo Registado': 'Consumo' + str(i)}, inplace=True)
    print(df.shape)
    i += 1
df

../Our_Dataset/PT0002000110339041CV.xlsx
(75460, 2)
Time range: 2020-10-06 00:15:00 - 2022-12-01 00:00:00
(75460, 2)
../Our_Dataset/PT0002000009906944AL.xlsx
(39268, 2)
Time range: 2021-10-19 00:15:00 - 2022-12-02 00:00:00
(75572, 3)
../Our_Dataset/PT0002000052213942JN.xlsx
../Our_Dataset/PT0002000105307115VP.xlsx
(37920, 2)
Time range: 2021-11-01 00:15:00 - 2022-12-01 00:00:00
(75588, 4)
../Our_Dataset/PT0002000051290369QG.xlsx
(37060, 2)
Time range: 2021-10-11 00:15:00 - 2022-11-01 00:00:00
(75636, 5)
../Our_Dataset/PT0002000071112543QL.xlsx
(69216, 2)
Time range: 2021-01-01 00:15:00 - 2022-12-23 00:00:00
(77748, 6)
../Our_Dataset/PT0002000083005987ZW.xlsx
(74888, 2)
Time range: 2020-10-06 00:15:00 - 2022-11-25 01:00:00
(77948, 7)


,Time,Consumo0,Consumo1,Consumo3,Consumo4,Consumo5,Consumo6
0,2020-10-06 00:15:00,0.076,NaN,NaN,NaN,NaN,0.320
1,2020-10-06 00:30:00,0.072,NaN,NaN,NaN,NaN,0.308
2,2020-10-06 00:45:00,0.072,NaN,NaN,NaN,NaN,0.292
3,2020-10-06 01:00:00,0.068,NaN,NaN,NaN,NaN,0.280
4,2020-10-06 01:15:00,0.032,NaN,NaN,NaN,NaN,0.340
...,...,...,...,...,...,...,...
77943,2022-12-22 23:00:00,NaN,NaN,NaN,NaN,0.424,NaN
77944,2022-12-22 23:15:00,NaN,NaN,NaN,NaN,0.364,NaN
77945,2022-12-22 23:30:00,NaN,NaN,NaN,NaN,0.360,NaN
77946,2022-12-22 23:45:00,NaN,NaN,NaN,NaN,0.312,NaN


In [30]:
df_sync = df.dropna()
df_sync.reset_index(inplace=True, drop=True)
print("Time range" + ": " + str(min(df_sync['Time'])) + " - " + str(max(df_sync['Time'])))
df_sync

Time range: 2021-11-01 00:15:00 - 2022-11-01 00:00:00


,Time,Consumo0,Consumo1,Consumo3,Consumo4,Consumo5,Consumo6
0,2021-11-01 00:15:00,0.028,0.452,0.000,0.828,0.128,0.552
1,2021-11-01 00:30:00,0.080,0.440,0.000,0.796,0.160,0.492
2,2021-11-01 00:45:00,0.092,0.424,0.000,0.832,0.220,0.308
3,2021-11-01 01:00:00,0.040,0.408,0.000,0.800,0.404,0.156
4,2021-11-01 01:15:00,0.076,0.396,0.000,0.836,0.156,0.160
...,...,...,...,...,...,...,...
35283,2022-10-31 23:00:00,0.120,0.448,0.108,0.220,0.164,0.368
35284,2022-10-31 23:15:00,0.036,0.424,0.124,0.252,0.228,0.396
35285,2022-10-31 23:30:00,0.064,0.392,0.148,0.268,0.224,0.356
35286,2022-10-31 23:45:00,0.132,0.260,0.132,0.276,0.212,0.392


In [42]:
dplctd = df_sync.duplicated(subset=['Time']).sum()
dplctd

252

In [43]:
df.sort_values(by=['Time'], inplace=True)
df

,Time,Consumo0,Consumo1,Consumo3,Consumo4,Consumo5,Consumo6
0,2020-10-06 00:15:00,0.076,NaN,NaN,NaN,NaN,0.320
1,2020-10-06 00:30:00,0.072,NaN,NaN,NaN,NaN,0.308
2,2020-10-06 00:45:00,0.072,NaN,NaN,NaN,NaN,0.292
3,2020-10-06 01:00:00,0.068,NaN,NaN,NaN,NaN,0.280
4,2020-10-06 01:15:00,0.032,NaN,NaN,NaN,NaN,0.340
...,...,...,...,...,...,...,...
77943,2022-12-22 23:00:00,NaN,NaN,NaN,NaN,0.424,NaN
77944,2022-12-22 23:15:00,NaN,NaN,NaN,NaN,0.364,NaN
77945,2022-12-22 23:30:00,NaN,NaN,NaN,NaN,0.360,NaN
77946,2022-12-22 23:45:00,NaN,NaN,NaN,NaN,0.312,NaN


In [44]:
for feature in df_sync.columns:
    print(feature + ": " + str(min(df_sync[feature])) + " - " + str(max(df_sync[feature])))

Time: 2021-11-01 00:15:00 - 2022-11-01 00:00:00
Consumo0: 0.0 - 3.172
Consumo1: 0.0 - 6.076
Consumo3: 0.0 - 3.26
Consumo4: 0.0 - 3.58
Consumo5: 0.04 - 4.856
Consumo6: 0.0 - 5.656


In [45]:
df_sync.to_excel("output.xlsx")